In [73]:
import nidaqmx
from nidaqmx.constants import LineGrouping,Edge, AcquisitionType, VoltageUnits, TerminalConfiguration, READ_ALL_AVAILABLE
# from nidaqmx import stream_readers
from nidaqmx.stream_writers import DigitalSingleChannelWriter, DigitalMultiChannelWriter

import numpy as np

In [2]:
system = nidaqmx.system.System.local()  # load local system

task_names = system.tasks.task_names  # returns a list of task names

In [ ]:
digitalInputTask = nidaqmx.Task()
digitalInputTask.di_channels.add_di_chan("/Dev1/port0/line14")
digitalInputTask.timing.cfg_samp_clk_timing(1000, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=50)
di_trig = digitalInputTask.triggers.start_trigger
di_trig.cfg_dig_edge_start_trig("/Dev1/PFI9", Edge.RISING)

# # scanning for axis 1
samplenum = 36
samplerate = 3
# dotask = nidaqmx.Task()
# dotask.do_channels.add_do_chan("/Dev1/port0/line8")
# dotask.timing.cfg_samp_clk_timing(samplerate, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=samplenum)
# do_trig = dotask.triggers.start_trigger
# do_trig.cfg_dig_edge_start_trig("/Dev1/PFI9", Edge.RISING)
# dowritelist = np.array([0b000000000, 0b100000000]*int(samplenum/2), dtype=np.uint32)
# dowritestream = DigitalSingleChannelWriter(dotask.out_stream, auto_start=False)

# written = dowritestream.write_many_sample_port_uint32(dowritelist, timeout=10)


# scanning for axis 2----------------

dotask2 = nidaqmx.Task()
dotask2.do_channels.add_do_chan("/Dev1/port0/line8:12")
dotask2.timing.cfg_samp_clk_timing(samplerate, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=2*samplenum)
do_trig2 = dotask2.triggers.start_trigger
do_trig2.cfg_dig_edge_start_trig("/Dev1/PFI9", Edge.RISING)

# twdolist = [0b0000000000000, 0b1000000000000]*int(samplenum) + [0b000000000, 0b000000000, 0b100000000, 0b100000000]*int(samplenum/2)
twdolist = np.array([0b0000000000000, 0b0000000000000, 0b1000000000000, 0b1000000000000]*int(samplenum/2), dtype=np.uint32) + \
           np.array([0b000000000, 0b000000000, 0b000000000, 0b100000000, 0b100000000, 0b100000000]*int(samplenum/3), dtype=np.uint32)
dowritelist = np.array(twdolist, dtype=np.uint32)
# dowritelist = np.array([0b00000000000000, 0b10000000000000]*int(samplenum/2), dtype=np.uint32)
dowritestream2 = DigitalSingleChannelWriter(dotask2.out_stream, auto_start=False)

written = dowritestream2.write_many_sample_port_uint32(dowritelist, timeout=10)


# dotask.start()
dotask2.start()

data = digitalInputTask.read(50)
# dotask.wait_until_done(timeout=samplenum/samplerate*100)
dotask2.wait_until_done(timeout=samplenum2/samplerate2*100)
# dotask.stop()
dotask2.stop()
digitalInputTask.stop()

print("finished la?")
print(data)

# print(data)


In [ ]:
dotask.close()
dotask2.close()
digitalInputTask.close()


In [78]:
# hardware conneciton setting 
fastaxis = "x"
if fastaxis == "x":
    ch_forw_fa = 8 # P0 digital channel for forward motion of fast axis
    ch_back_fa = 9
    ch_forw_sa = 10
    ch_back_sa = 11 # P0 digital channel for backward motion of slow axis
elif fastaxis == "y":
    ch_forw_fa = 10 
    ch_back_fa = 11
    ch_forw_sa = 8
    ch_back_sa = 9
NI_ch_APD = "/Dev1/ai16"
NI_ch_Clock = "/Dev1/PFI8" # clock source
NI_ch_Trig = "/Dev1/PFI9" # clock gate
NI_ch_XYZstageTrig = "/Dev1/port0/line8:13"

# signal reading parameters
min_volt = -10 # [V]
max_volt = 10
samplerate_read = 0.1E6 # 500kHz .max ext clock rate of NI6343
timeout_read = 10
# scan parameters
rate_slipstick = 2099E3 # [mHz],  from 1Hz to 5kHz
stepsperpulse = 20
basetime = 1.0/samplerate_read
dutyportion = 0.1
samplerate_scan = (dutyportion * (rate_slipstick/1000.0)/stepsperpulse)
assert samplerate_read >= samplerate_scan
samplerate_scan = 1.0/(basetime*round((1.0/samplerate_scan)//basetime))
num_fastaxis = 6
num_slowaxis = 6
ttl_forw_line_fa = [0, 2**ch_forw_fa]*num_fastaxis # use rasing edge to trigger
ttl_idle_fa = [0, 0]*num_fastaxis
ttl_back_line_fa = [0, 2**ch_back_fa]*num_fastaxis
datanum_per_scanpoint = int(samplerate_read/samplerate_scan)
print(datanum_per_scanpoint)

# num_scansample = 4*num_fastaxis*2*num_slowaxis
# scantime = num_scansample/samplerate
# print(f"Estimated Scanning Time:{scantime} s")
# ttl_scan_fa = np.array((ttl_forw_line_fa+ttl_idle_fa+ttl_back_line_fa+ttl_idle_fa)*num_slowaxis, dtype=np.uint32)
# ttl_scan_sa = np.array(([2**ch_forw_sa]*4*num_fastaxis+[0]*4*num_fastaxis)*num_slowaxis, dtype=np.uint32)

num_scansample = 2*num_fastaxis*2*num_slowaxis
scantime = num_scansample/samplerate
print(f"Estimated Scanning Time:{scantime} s")
ttl_scan_fa = np.array((ttl_forw_line_fa+ttl_back_line_fa)*num_slowaxis, dtype=np.uint32)
ttl_scan_sa = np.array(([2**ch_forw_sa]*2*num_fastaxis+[0]*2*num_fastaxis)*num_slowaxis, dtype=np.uint32)
dotowrite = ttl_scan_fa + ttl_scan_sa


# num_readsample = num_scansample*datanum_per_scanpoint
# buffer_read = np.zeros(num_readsample, dtype=np.float64, order='C')
# buffer_read = np.zeros(num_readsample, dtype=np.float64, order='C')

# scantask = nidaqmx.Task("scanner")
# readtask = nidaqmx.Task("readsignal")


# try:
#     scantask.do_channels.add_do_chan(NI_ch_XYZstageTrig)
#     scantask.timing.cfg_samp_clk_timing(2*samplerate, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_scansample)
#     scan_trig = scantask.triggers.start_trigger
#     scan_trig.cfg_dig_edge_start_trig("/Dev1/PFI9", Edge.RISING)
#     writestream = DigitalSingleChannelWriter(scantask.out_stream, auto_start=False)
#     written = writestream.write_many_sample_port_uint32(dotowrite, timeout=10)

#     readtask.ai_channels.add_ai_voltage_chan(
#                 NI_ch_APD,"",
#                 # TerminalConfiguration.RSE,
#                 TerminalConfiguration.DIFF,
#                 min_volt,max_volt,
#                 VoltageUnits.VOLTS
#             )
#     readtask.timing.cfg_samp_clk_timing(samplerate_read, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_readsample)
#     read_trig = readtask.triggers.start_trigger
#     read_trig.cfg_dig_edge_start_trig(NI_ch_Trig, Edge.RISING)
#     reader = nidaqmx.stream_readers.AnalogSingleChannelReader(readtask.in_stream)
#             # reader = stream_readers.AnalogMultiChannelReader(task.in_stream)
#     reader.read_all_avail_samp  = True

#     scantask.start() #start scanning
#     num_read = reader.read_many_sample(
#             buffer_read,
#             READ_ALL_AVAILABLE,
#             timeout_read
#         )
#     scantask.wait_until_done(timeout=scantime*10) # block the code below, optional
#     scantask.stop()
# except Exception as ee:
#     print(ee)
# finally:
#     scantask.close()
#     readtask.close()
# print(buffer_read)


ZeroDivisionError: float division by zero

## 1D scan 
come back to starting pointing point after each line


In [ ]:
# hardware conneciton setting 
fastaxis = "x"
if fastaxis == "x":
    ch_forw_fa = 8 # P0 digital channel for forward motion of fast axis
    ch_back_fa = 9
    ch_forw_sa = 10
    ch_back_sa = 11 # P0 digital channel for backward motion of slow axis
elif fastaxis == "y":
    ch_forw_fa = 10 
    ch_back_fa = 11
    ch_forw_sa = 8
    ch_back_sa = 9

# scan parameters
rate_slipstick = 2099E3 # [mHz],  from 1Hz to 5kHz
stepsperpulse = 20
dutyportion = 0.1
samplerate = dutyportion * (rate_slipstick/1000.0)/stepsperpulse * 2

num_fastaxis = 66
num_slowaxis = 66
ttl_forw_line_fa = [0, 2**ch_forw_fa]*num_fastaxis # use rasing edge to trigger
ttl_idle_fa = [0, 0]*num_fastaxis
ttl_back_line_fa = [0, 2**ch_back_fa]*num_fastaxis

# num_scansample = 4*num_fastaxis*2*num_slowaxis
# scantime = num_scansample/samplerate
# print(f"Estimated Scanning Time:{scantime} s")
# ttl_scan_fa = np.array((ttl_forw_line_fa+ttl_idle_fa+ttl_back_line_fa+ttl_idle_fa)*num_slowaxis, dtype=np.uint32)
# ttl_scan_sa = np.array(([2**ch_forw_sa]*4*num_fastaxis+[0]*4*num_fastaxis)*num_slowaxis, dtype=np.uint32)

num_scansample = 2*num_fastaxis*2*num_slowaxis
scantime = num_scansample/samplerate
print(f"Estimated Scanning Time:{scantime} s")
ttl_scan_fa = np.array((ttl_forw_line_fa+ttl_back_line_fa)*num_slowaxis, dtype=np.uint32)
ttl_scan_sa = np.array(([2**ch_forw_sa]*2*num_fastaxis+[0]*2*num_fastaxis)*num_slowaxis, dtype=np.uint32)


dotowrite = ttl_scan_fa + ttl_scan_sa


scantask = nidaqmx.Task("scanner")
try:
    scantask.do_channels.add_do_chan("/Dev1/port0/line8:13")
    scantask.timing.cfg_samp_clk_timing(samplerate, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_scansample)
    scan_trig = scantask.triggers.start_trigger
    scan_trig.cfg_dig_edge_start_trig("/Dev1/PFI9", Edge.RISING)
    writestream = DigitalSingleChannelWriter(scantask.out_stream, auto_start=False)
    written = writestream.write_many_sample_port_uint32(dotowrite, timeout=10)

    scantask.start() #start scanning
    scantask.wait_until_done(timeout=scantime*10) # block the code below, optional
    scantask.stop()
except Exception as ee:
    print(ee)
finally:
    scantask.close()

In [21]:
scantask.close()